# New Section

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
stop = stopwords.words('english')

In [ ]:
df=pd.read_csv('data.csv',lineterminator='\n')

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import re

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['joint_review']=df['Review_Title']+df['Review']

In [ ]:
df['Sentiments']=np.where(df['Rating\r']>=4.0,1,0)

In [ ]:
df = df[['joint_review','Sentiments']]

In [ ]:
df['joint_review']=df['joint_review'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [ ]:
df['joint_review']=df['joint_review'].str.lower()

In [ ]:
df['joint_review']=df['joint_review'].apply(lambda x: ' '.join(i for i in x.split() if i not in stop) )

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = 50000
embedding_dim=16
max_length=50
trunc_type='post'
oov_tok="<oov>"

In [ ]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

In [ ]:
X=df['joint_review']
y=df['Sentiments']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok, split=' ')
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
padded= pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

In [ ]:
testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded= pad_sequences(testing_sequences,maxlen=max_length,truncating=trunc_type)

In [ ]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
    
])

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(padded,y_train,epochs=10,validation_data=(testing_padded,y_test))

In [ ]:
model.save('Car_NLP.h5')

In [ ]:
e=model.layers[0]
weights=e.get_weights()[0]
print(weights.shape)

In [ ]:
word_index=tokenizer.word_index

In [ ]:
reverse_word_index=dict([(value,key) for (key,value) in word_index.items()])

In [ ]:
import io
out_v=io.open('vecs2.tsv','w',encoding='utf-8')
out_m=io.open('meta2.tsv','w',encoding='utf-8')
for word_num in range(1,vocab_size):
   word=reverse_word_index[word_num]
   embedding=weights[word_num]
   out_m.write(word+"\n")
   out_v.write('\t'.join([str(x) for x in embedding])+"\n")
out_v.close()
out_m.close()

In [ ]:
value = ['fun & quality']
#vectorizing
value = tokenizer.texts_to_sequences(value)
#padding 
value = pad_sequences(value, maxlen=28, dtype='int32', value=0)
print(value)
sentiment = model.predict(value,batch_size=1,verbose = 2)[0]
print(sentiment)

In [ ]:
y_pred = model.predict_classes(value)
y_pred[0]
if y_pred[0]==[1]:
  print("positive")
else:
  print("negative")